<a href="https://colab.research.google.com/github/george-marcus/ml-neural-networks/blob/main/MLP_MakeMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import torch.nn.functional as F

In [3]:
import matplotlib as plt

In [4]:
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
words = open('/content/drive/MyDrive/Colab Notebooks/names.txt', 'r').read().splitlines()

In [7]:
block_size = 3

In [8]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [9]:
X,Y = [],[]
for w in words:
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    context = context[1:] + [ix] # crop and append

In [10]:
X = torch.tensor(X)

In [11]:
Y = torch.tensor(Y)

In [15]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2),generator=g)
W1 = torch.randn((6,100),generator=g)
b1 = torch.randn(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
  p.requires_grad = True


In [17]:
lre = torch.linspace(-3,0,1000)
lrs = 10**lre

ix = torch.randint(0,X.shape[0],(32,))
for i in range(1000):
  # forward pass
  emb = C[X[ix]]
  h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits,Y[ix])
  #  backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  # update
  lr = lrs[i]
  for p in parameters:
    p.data += -lr * p.grad

In [21]:
emb = C[X[ix]]
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Y[ix])
loss

tensor(0.0020, grad_fn=<NllLossBackward0>)